# example code for checking installation

In [ ]:
import pyomo.environ as pyo
from pyomo2h5 import save_h5

# Create Model
m = pyo.ConcreteModel()

m.I = pyo.RangeSet(5)

m.p = pyo.Param(initialize=5,doc="this is a parameter")

m.x = pyo.Var(m.I, within=pyo.NonNegativeReals,doc="this is an indexed variable")

@m.Objective(doc="This is an Objective", sense=pyo.minimize)
def obj(m):
    return sum(m.x[i] for i in m.I)

@m.Constraint(doc="This is a Constraint")
def con1(m):
    return m.x[1] >= m.p

# Solve Model
filename = "test6"

solver = pyo.SolverFactory("gurobi", solver_io="python")
solver.options["LogFile"] = filename + ".log"
results = solver.solve(m, tee=True)

# Save the dictionary to an HDF5 file
save_h5(m, results, filename,
        save_log_flag=False,
        save_constraint_flag=True,
        pickle_flag=True,
        git_flag=True)
print("Dictionary saved to HDF5 file successfully!")